In [7]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import os
import torchvision.transforms as transforms
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision.transforms import Compose
from torchvision.transforms import Resize
import torch.optim as optim
from torch.utils.data import DataLoader , TensorDataset
from torch import Tensor
#!pip install einops
!pip install vit-pytorch
from torchvision.models import resnet18
#from vit_pytorch.mobile_vit import MobileViT
from vit_pytorch.vit_for_small_dataset import ViT


    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

ModuleNotFoundError: No module named 'mobilevit_pytorch'

In [2]:
from numpy import load
from sklearn.model_selection import train_test_split

from google.colab import drive
drive.mount('/content/drive')

# images = load('/content/drive/My Drive/SYDE675_Leaf_Disease_Classification/KL_data/x_train.npy')
# labels = load('/content/drive/My Drive/SYDE675_Leaf_Disease_Classification/KL_data/y_train.npy')

######### KL colab
images = load('/content/drive/MyDrive/Colab_data/data_256/x_train.npy')
labels = load('/content/drive/MyDrive/Colab_data/data_256/y_train.npy')

# images = load('x_train.npy')
# labels = load('y_train.npy')



x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
x_train = x_train.reshape((-1,3,256,256))
x_test = x_test.reshape((-1,3,256,256))

In [3]:

print(np.shape(x_train))

(2880, 3, 256, 256)


In [4]:
torch.cuda.is_available()

False

In [5]:



image_size = 256

# Define transformations
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop((256, 256), scale=(0.7, 1.0), ratio=(0.9, 1.1)),  # Random crop and resize
    transforms.RandomRotation(degrees=(-10, 10), fill=(0,0,0)),  # Random rotation with black fill
    transforms.ColorJitter(brightness=0.1, contrast=0.5, saturation=0.1),  # Random color adjustments
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop((256, 256), scale=(0.7, 1.0), ratio=(0.9, 1.1)),  # Random crop and resize
    transforms.RandomRotation(degrees=(-10, 10), fill=(0,0,0)),  # Random rotation with black fill
    transforms.ColorJitter(brightness=0.1, contrast=0.5, saturation=0.1),  # Random color adjustments
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])



# Apply transformations to custom datasets
x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
x_train_processed = torch.stack([train_transform(image) for image in x_train_tensor])
x_test_processed = torch.stack([val_transform(image) for image in x_test_tensor])


# Create DataLoader objects
batch_size = 64

dataset_train = TensorDataset( x_train_processed, Tensor(y_train).long() )
dataset_test = TensorDataset( x_test_processed, Tensor(y_test).long())

train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)




In [ ]:
# Initialize model, loss, and optimizer
num_epochs = 300

train_loss_rec = np.zeros(num_epochs)
train_acc_rec =np.zeros(num_epochs) 
test_acc_rec = np.zeros(num_epochs)
# model  = MobileViT(
#     image_size = (256, 256),
#     dims = [96, 120, 144],
#     channels = [16, 32, 48, 48, 64, 64, 80, 80, 96, 96, 384],
#     num_classes = 2).to(device)
model = ViT(
    image_size = 256,
    patch_size = 16,
    num_classes = 1000,
    dim = 1024,
    depth = 6,
    heads = 16,
    mlp_dim = 2048,
    dropout = 0.1,
    emb_dropout = 0.1).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.RAdam(model.parameters(), lr=1e-4, weight_decay=2e-2)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")
    train_loss_rec[epoch] = running_loss/len(train_loader)
    # Evaluation
    model.eval()
    train_correct = 0
    train_total = 0
    test_correct = 0
    test_total = 0
    with torch.no_grad():
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()


        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()
    
    print(f"train_Accuracy: {100 * train_correct / train_total}%")
    print(f"test_Accuracy: {100 * test_correct / test_total}%")
    train_acc_rec[epoch] =train_correct / train_total 
    test_acc_rec[epoch] =test_correct / test_total

In [ ]:
x_test = load('/content/drive/My Drive/SYDE675_Leaf_Disease_Classification/KL_data/x_test.npy')
y_test = load('/content/drive/My Drive/SYDE675_Leaf_Disease_Classification/KL_data/y_test.npy')

x_test = x_test.reshape((-1,3,256,256))
x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
x_test_processed = torch.stack([test_transform(image) for image in x_test_tensor])

dataset_test = TensorDataset( x_test, Tensor(y_test).long())
test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)

In [ ]:
 # Evaluation
model.eval()

test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()
    
test_acc =test_correct / test_total

In [ ]:
from numpy import save

save('/content/drive/My Drive/SYDE675_Leaf_Disease_Classification/KL_data/Mvit_noise_train_loss_rec.npy', train_loss_rec)
save('/content/drive/My Drive/SYDE675_Leaf_Disease_Classification/KL_data/Mvit_noise_train_acc_rec.npy', train_acc_rec)
save('/content/drive/My Drive/SYDE675_Leaf_Disease_Classification/KL_data/Mvit_noise_val_acc_rec.npy', test_acc_rec)
save('/content/drive/My Drive/SYDE675_Leaf_Disease_Classification/KL_data/Mvit_noise_test_acc.npy', test_acc)